In [1]:
! pip install mordred
! pip install pubchempy
! pip install -U scikit-learn

DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\absl_py-1.4.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\astunparse-1.6.3-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\cachetools-5.3.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\flatbuffers-23.5.26-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package ins

DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\absl_py-1.4.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\astunparse-1.6.3-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\cachetools-5.3.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\supak\appdata\local\programs\python\python311\lib\site-packages\flatbuffers-23.5.26-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package ins

In [20]:
import pandas as pd
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from mordred import Calculator, descriptors
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("data_compound.csv")
df

,smiles,activity,type,value (nM),comment,sid,cid
0,C[C@H](C(=O)N1CCC2=C(C1)N=C(C(=N2)NCC(F)F)N3CC...,Active,IC50,0.0066,NaN,316936203,118159182
1,C[C@@H](C(=O)N1CCC2=C(C1)N=C(C(=N2)NCC(F)F)N3C...,Active,IC50,0.0083,NaN,316936247,118159176
2,CC(C)NC1=NC2=C(CN(CC2)C(=O)C)N=C1N3CCC(CC3)C(C...,Active,IC50,0.0085,NaN,316936221,118159272
3,CC(C)NC1=NC2=C(CN(CC2)C)N=C1N3CCC(CC3)C(=O)C4=...,Active,IC50,0.0088,NaN,316936228,118159276
4,CN(C)C(=O)N1CCC2=C(C1)N=C(C(=N2)NCC(F)F)N3CCC(...,Active,IC50,0.0091,NaN,316936201,118159362
...,...,...,...,...,...,...,...
1590,CC(C)NC1=NC(=CN=C1N2CCC(CC2)OC3=C(C=C(C=C3)F)F...,Unspecified,EC50,242000.0000,Outside typical range,440225257,118309468
1591,CC1C2=C(CCN1C(=O)C3CC3)N=C(C(=N2)N4CCC(CC4)OC5...,Unspecified,IC50,259000.0000,Outside typical range,374317775,118159258
1592,CC1C2=C(CCN1C(=O)C3CC3)N=C(C(=N2)N4CCC(CC4)OC5...,Unspecified,IC50,259000.0000,Outside typical range,440114933,155513954
1593,CC(C)NC1=NC2=C(CCN(C2)C(=O)N3CCCC3)N=C1N4CCC(C...,Unspecified,IC50,391000.0000,Outside typical range,374288237,118159228


In [4]:
import pubchempy as pcp

In [5]:
def canonical_smiles(smiles):
    mol_smiles = [Chem.MolFromSmiles(smi) for smi in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mol_smiles]

    return smiles

In [6]:
CANNO_SMILES = canonical_smiles(df.smiles)
len(CANNO_SMILES)

1595

In [7]:
df['smiles'] = CANNO_SMILES
df

,smiles,activity,type,value (nM),comment,sid,cid
0,CO[C@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc...,Active,IC50,0.0066,NaN,316936203,118159182
1,CO[C@@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4cc...,Active,IC50,0.0083,NaN,316936247,118159176
2,CC(=O)N1CCc2nc(NC(C)C)c(N3CCC(C(F)c4ccc(F)cc4F...,Active,IC50,0.0085,NaN,316936221,118159272
3,CC(C)Nc1nc2c(nc1N1CCC(C(=O)c3cc(Cl)ccc3F)CC1)C...,Active,IC50,0.0088,NaN,316936228,118159276
4,CN(C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc...,Active,IC50,0.0091,NaN,316936201,118159362
...,...,...,...,...,...,...,...
1590,CC(C)Nc1nc(C(=O)N(C)C)cnc1N1CCC(Oc2ccc(F)cc2F)CC1,Unspecified,EC50,242000.0000,Outside typical range,440225257,118309468
1591,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)C(C)N(...,Unspecified,IC50,259000.0000,Outside typical range,374317775,118159258
1592,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)C(C)N(...,Unspecified,IC50,259000.0000,Outside typical range,440114933,155513954
1593,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)CCN(C(...,Unspecified,IC50,391000.0000,Outside typical range,374288237,118159228


In [8]:
smiles_list = list(df["smiles"])
smiles_list

['CO[C@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc4F)CC3)nc2C1',
 'CO[C@@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc4F)CC3)nc2C1',
 'CC(=O)N1CCc2nc(NC(C)C)c(N3CCC(C(F)c4ccc(F)cc4F)CC3)nc2C1',
 'CC(C)Nc1nc2c(nc1N1CCC(C(=O)c3cc(Cl)ccc3F)CC1)CN(C)CC2',
 'CN(C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc4F)CC3)nc2C1',
 'CN1CCc2nc(NC3CC3)c(N3CCC(C(F)c4ccc(F)cc4F)CC3)nc2C1',
 'CC(=O)N1CCc2nc(NC(C)C)c(N3CCC(Oc4ccc(F)cc4F)CC3)nc2C1',
 'CC(=O)N1CCc2nc(NC(C)C)c(N3CCC(C(=O)c4cc(Cl)ccc4F)CC3)nc2C1',
 'CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)CCN(C(=O)N(C)C)C2',
 'CC1Cc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc4F)CC3)nc2CN1S(C)(=O)=O',
 'CC(C)Nc1nc2c(nc1N1CCC([C@H](F)c3ccc(F)cc3F)CC1)CCN(C=O)C2',
 'CC(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc4F)CC3)nc2C1',
 'COc1ccc(OC2CCN(c3nc4c(nc3NC3CCC3)CCN(C(C)=O)C4)CC2)c(F)c1',
 'CC(=O)N1CCc2nc(N3CCC(C(F)c4ccc(F)cc4F)CC3)c(NC(C)C)nc2C1',
 'CC(C)Nc1nc2c(nc1N1CCC(C(=O)c3cc(Cl)ccc3F)CC1)CCN(C)C2',
 'CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)CN(C(=O)N(C)C)CC2C',
 'CC(=O)N1CCc

**RDkit Descriptor**

In [9]:
smiles2 = [Chem.MolFromSmiles(i) for i in smiles_list]
mols2 = [m for m in smiles2]
len(mols2)

1595

In [10]:
alldesc = [Descriptors.CalcMolDescriptors(m) for m in mols2]
rdkit_gpr6 = pd.DataFrame(alldesc)
rdkit_gpr6

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.972261,13.972261,0.015965,-2.565677,0.544559,511.520,482.288,511.220653,196,0,...,0,0,0,0,0,0,0,0,0,0
1,13.972261,13.972261,0.015965,-2.565677,0.544559,511.520,482.288,511.220653,196,0,...,0,0,0,0,0,0,0,0,0,0
2,15.079211,15.079211,0.010379,-1.486341,0.715021,461.532,431.292,461.240245,178,0,...,0,0,0,0,0,0,0,0,0,0
3,14.162533,14.162533,0.087707,-0.509331,0.696577,445.970,416.738,445.204466,166,0,...,0,0,0,0,0,0,0,0,0,0
4,13.954396,13.954396,0.010776,-2.556825,0.617440,496.509,468.285,496.220987,190,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,13.848314,13.848314,0.058663,-0.701052,0.774676,419.476,392.260,419.213282,162,0,...,0,0,0,0,0,0,0,0,0,0
1591,14.030641,14.030641,0.078548,-0.684547,0.650805,485.579,452.315,485.260232,188,0,...,0,0,0,0,0,0,0,0,0,0
1592,14.030641,14.030641,0.078548,-5.083333,0.441450,599.601,565.329,599.253096,230,0,...,0,0,0,0,0,0,0,0,0,0
1593,14.017818,14.017818,0.082053,-0.681796,0.663656,500.594,466.322,500.271131,194,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
#df_gpr6_compound = df.join(df_descrs)
#df_gpr6_compound

In [19]:
df_smiles = df[['smiles', 'activity']]
df_merged_smiles_rdkit = df_smiles.join(rdkit_gpr6)
df_merged_smiles_rdkit

,smiles,activity,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CO[C@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc...,Active,13.972261,13.972261,0.015965,-2.565677,0.544559,511.520,482.288,511.220653,...,0,0,0,0,0,0,0,0,0,0
1,CO[C@@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4cc...,Active,13.972261,13.972261,0.015965,-2.565677,0.544559,511.520,482.288,511.220653,...,0,0,0,0,0,0,0,0,0,0
2,CC(=O)N1CCc2nc(NC(C)C)c(N3CCC(C(F)c4ccc(F)cc4F...,Active,15.079211,15.079211,0.010379,-1.486341,0.715021,461.532,431.292,461.240245,...,0,0,0,0,0,0,0,0,0,0
3,CC(C)Nc1nc2c(nc1N1CCC(C(=O)c3cc(Cl)ccc3F)CC1)C...,Active,14.162533,14.162533,0.087707,-0.509331,0.696577,445.970,416.738,445.204466,...,0,0,0,0,0,0,0,0,0,0
4,CN(C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc...,Active,13.954396,13.954396,0.010776,-2.556825,0.617440,496.509,468.285,496.220987,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,CC(C)Nc1nc(C(=O)N(C)C)cnc1N1CCC(Oc2ccc(F)cc2F)CC1,Unspecified,13.848314,13.848314,0.058663,-0.701052,0.774676,419.476,392.260,419.213282,...,0,0,0,0,0,0,0,0,0,0
1591,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)C(C)N(...,Unspecified,14.030641,14.030641,0.078548,-0.684547,0.650805,485.579,452.315,485.260232,...,0,0,0,0,0,0,0,0,0,0
1592,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)C(C)N(...,Unspecified,14.030641,14.030641,0.078548,-5.083333,0.441450,599.601,565.329,599.253096,...,0,0,0,0,0,0,0,0,0,0
1593,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)CCN(C(...,Unspecified,14.017818,14.017818,0.082053,-0.681796,0.663656,500.594,466.322,500.271131,...,0,0,0,0,0,0,0,0,0,1


In [23]:
df_merged_smiles_rdkit.to_csv("rdkit_descriptors_gpr6.csv", index=False)

**Mordred Descriptors**

In [12]:
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=True)
    smiles3 = [Chem.MolFromSmiles(i) for i in data]
    
    df_mordred = calc.pandas(smiles3)
    return df_mordred

In [13]:
mordred_gpr6 = All_Mordred_descriptors(smiles_list)
mordred_gpr6.shape


(1595, 1613)

In [14]:
mordred_gpr6

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,27.990863,20.208094,0,0,46.356073,2.463468,4.926935,46.356073,1.287669,4.505313,...,10.488186,72.610895,511.220653,7.864933,4404,59,188.0,220.0,12.444444,7.916667
1,27.990863,20.208094,0,0,46.356073,2.463468,4.926935,46.356073,1.287669,4.505313,...,10.488186,72.610895,511.220653,7.864933,4404,59,188.0,220.0,12.444444,7.916667
2,25.938492,19.114455,0,0,41.715306,2.464737,4.929475,41.715306,1.2641,4.426747,...,10.479904,69.390362,461.240245,7.321274,3374,56,176.0,208.0,11.694444,7.111111
3,24.455329,18.151364,0,1,39.965051,2.458558,4.917115,39.965051,1.289195,4.367884,...,10.421954,67.098280,445.204466,7.420074,2814,52,166.0,196.0,10.583333,6.666667
4,27.393146,19.733756,0,0,44.686795,2.462679,4.925359,44.686795,1.276766,4.479689,...,10.470136,71.509172,496.220987,7.876524,4064,57,184.0,215.0,12.194444,7.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,23.190945,17.680679,0,0,37.427603,2.414824,4.829648,37.427603,1.247587,4.315473,...,10.244059,65.536305,419.213282,7.354619,2733,47,154.0,178.0,11.222222,6.555556
1591,27.909982,20.230577,0,0,45.202031,2.503941,4.994198,45.202031,1.291487,4.514583,...,10.604826,90.270707,485.260232,7.136180,3969,60,192.0,230.0,11.305556,7.472222
1592,32.786549,25.440429,1,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,10.716216,97.934664,599.253096,7.884909,24500004011,66,222.0,260.0,16.479167,8.972222
1593,28.588580,20.347975,0,0,46.796718,2.468206,4.933569,46.796718,1.299909,4.522784,...,10.531056,85.920761,500.271131,7.146730,4474,58,194.0,229.0,10.944444,7.750000


In [15]:
mordred_gpr6.iloc[1592]

ABC                                  32.786549
ABCGG                                25.440429
nAcid                                        1
nBase                                        0
SpAbs_A     multiple fragments (SpAbs_A/SpAbs)
                           ...                
WPol                                        66
Zagreb1                                  222.0
Zagreb2                                  260.0
mZagreb1                             16.479167
mZagreb2                              8.972222
Name: 1592, Length: 1613, dtype: object

In [22]:
df_merged_smiles_mordred = df_smiles.join(mordred_gpr6)
df_merged_smiles_mordred

,smiles,activity,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,CO[C@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc...,Active,27.990863,20.208094,0,0,46.356073,2.463468,4.926935,46.356073,...,10.488186,72.610895,511.220653,7.864933,4404,59,188.0,220.0,12.444444,7.916667
1,CO[C@@H](C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4cc...,Active,27.990863,20.208094,0,0,46.356073,2.463468,4.926935,46.356073,...,10.488186,72.610895,511.220653,7.864933,4404,59,188.0,220.0,12.444444,7.916667
2,CC(=O)N1CCc2nc(NC(C)C)c(N3CCC(C(F)c4ccc(F)cc4F...,Active,25.938492,19.114455,0,0,41.715306,2.464737,4.929475,41.715306,...,10.479904,69.390362,461.240245,7.321274,3374,56,176.0,208.0,11.694444,7.111111
3,CC(C)Nc1nc2c(nc1N1CCC(C(=O)c3cc(Cl)ccc3F)CC1)C...,Active,24.455329,18.151364,0,1,39.965051,2.458558,4.917115,39.965051,...,10.421954,67.098280,445.204466,7.420074,2814,52,166.0,196.0,10.583333,6.666667
4,CN(C)C(=O)N1CCc2nc(NCC(F)F)c(N3CCC(Oc4ccc(F)cc...,Active,27.393146,19.733756,0,0,44.686795,2.462679,4.925359,44.686795,...,10.470136,71.509172,496.220987,7.876524,4064,57,184.0,215.0,12.194444,7.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,CC(C)Nc1nc(C(=O)N(C)C)cnc1N1CCC(Oc2ccc(F)cc2F)CC1,Unspecified,23.190945,17.680679,0,0,37.427603,2.414824,4.829648,37.427603,...,10.244059,65.536305,419.213282,7.354619,2733,47,154.0,178.0,11.222222,6.555556
1591,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)C(C)N(...,Unspecified,27.909982,20.230577,0,0,45.202031,2.503941,4.994198,45.202031,...,10.604826,90.270707,485.260232,7.136180,3969,60,192.0,230.0,11.305556,7.472222
1592,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)C(C)N(...,Unspecified,32.786549,25.440429,1,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),...,10.716216,97.934664,599.253096,7.884909,24500004011,66,222.0,260.0,16.479167,8.972222
1593,CC(C)Nc1nc2c(nc1N1CCC(Oc3ccc(F)cc3F)CC1)CCN(C(...,Unspecified,28.588580,20.347975,0,0,46.796718,2.468206,4.933569,46.796718,...,10.531056,85.920761,500.271131,7.146730,4474,58,194.0,229.0,10.944444,7.750000


In [24]:
df_merged_smiles_mordred.to_csv("mordred_descriptors_gpr6.csv", index=False)

**Clustering**